In [ ]:
import gym
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

In [ ]:
env = gym.make('CartPole-v1')
env.reset()

## Defining TRPO setup

In [ ]:
def var_size(v):
    return int(np.prod([int(d) for d in v.shape]))

def get_padded_gradients(loss, var_list):
    grads = tf.gradients(loss, var_list)
    return [g if g is not None else tf.zeros(v.shape)
            for g, v in zip(grads, var_list)]

def get_flattened_gradients(loss, var_list):
    padded_gradients = get_padded_gradients(loss, var_list)
    return tf.concat([tf.reshape(x, [-1]) for x in padded_gradients], 0)

def unflatten_gradient(grad, var_list):
    shapes = [v.shape for v in var_list]
    sizes = [var_size(v) for v in var_list]
    grads = []

    pointer = 0
    for (shape, size, v) in zip(shapes, sizes, var_list):
        grads.append(tf.reshape(grad[pointer:pointer + size], shape))
        pointer += size
    return grads

def sum_discounted_rewards(rewards, discount):
    discounted_rewards = list(rewards)
    pointer = len(rewards) - 1
    acc_discounted_sum = rewards[-1]
    while pointer > 0:
        acc_discounted_sum *= 0.95
        pointer -= 1
        discounted_rewards[pointer] += acc_discounted_sum
        acc_discounted_sum += rewards[pointer]
    return discounted_rewards

In [ ]:
def cg_solver(Ax_fun, b, k_iter=10, eps=10**(-6)):
    # Solver for Ax = b that uses conjugate gradient method
    # Ax_fun is supposed to be a function that takes x and returns Ax
    # Impementation of the solver is taken from Wikipedia article
    # Solver itself is supposed to be outside tf realm - all vector inside are np arrays
    # Note that it is assumed k_iter < dim(b)
    x = np.zeros(b.shape, dtype=np.float)
    r = b - Ax_fun(x)
    p = np.array(r)
    for k in range(k_iter):
        
        r_norm = float(np.sum(r * r))
        A_p = Ax_fun(p)
        alpha = r_norm / np.sum(p * A_p)
        x += alpha * p
        r -= alpha * A_p
        next_r_norm = np.sum(r * r)
        
        if next_r_norm < eps:
            break
            
        beta = next_r_norm / r_norm
        p *= beta
        p += r
        
    return x   

In [ ]:
class RL_Agent:
    
    def __init__(self, model_name):
        with tf.variable_scope(model_name):
            self.model_name = model_name
            self.session = tf.Session()
            
            self.input_layer = tf.placeholder(shape=[None, 4], dtype=tf.float32)
            self.dense1_layer = tf.layers.dense(self.input_layer, 
                                                units=4, use_bias=True, 
                                                activation=tf.nn.relu, name="dense1_weights"
                                               )
            
            self.dense2_layer = tf.layers.dense(self.dense1_layer, 
                                                units=2, use_bias=True, 
                                                activation=tf.nn.relu, name="dense2_weights"
                                               ) 
            
            self.prob_layer = tf.maximum(tf.minimum(tf.nn.softmax(self.dense2_layer), 0.9999), 0.0001)
            self.log_prob_layer = tf.log(self.prob_layer)
                        
            self.session.run(tf.global_variables_initializer())

    def model_variables(self):
        return [x for x in tf.trainable_variables() if self.model_name in x.name]
    
    def model_size(self):
        var_sizes = [tf.size(x) for x in self.model_variables()]
        return self.session.run(tf.reduce_sum(var_sizes))
    
    def variable_size(self):
        var_sizes = [tf.size(x) for x in self.model_variables()]
        return self.session.run(var_sizes)
            
    def predict(self, states):
        return self.session.run(self.prob_layer, feed_dict={self.input_layer: states})
    
    def pg_grad(self, states, actions, rewards):
        # Calculating base policy gradient
        # Return a sum of log_prob gradients weighted by discounted sum of future rewards
        action_mask = tf.one_hot(actions, depth=2, on_value=1.0, off_value=0.0, axis=-1)
        picked_log_prob_actions = tf.reduce_sum(action_mask * self.log_prob_layer, axis=1)
        weighted_log_prob_actions = picked_log_prob_actions * rewards
        grad_log_prob_actions = get_flattened_gradients(weighted_log_prob_actions, self.model_variables())
        return self.session.run(grad_log_prob_actions, feed_dict={self.input_layer: states})

    def trpo_grad(self, states, actions, rewards):
        # Calculating the target gradient as defined in section 5 of the trpo paper
        # Takes a gradient of prob action ratio weighted by Q-values
        # Essentially it should be a gradient for compare_prob_ratios function
        action_mask = tf.one_hot(actions, depth=2, on_value=1.0, off_value=0.0, axis=-1)
        fixed_prob_actions = tf.stop_gradient(self.prob_layer)
        prob_ratio = self.prob_layer / fixed_prob_actions
        masked_prob_ratio = tf.reduce_sum(action_mask * prob_ratio, axis=1)
        weighted_prob_ratio = masked_prob_ratio * rewards
        trpo_gradient = get_flattened_gradients(weighted_prob_ratio, self.model_variables())
        return self.session.run(trpo_gradient, feed_dict={self.input_layer: states})
    
    def fisher_vector_product(self, states, vector):
        # This function is supposed to return the product of estimated fisher information matrix and a specified vector
        # As I hope to reliably estimate this matrix, I take all states accumulated in a batch of games
        expected_log_prob = tf.reduce_sum(tf.stop_gradient(self.prob_layer) * self.log_prob_layer, axis=1)
        log_prob_grad = get_flattened_gradients(expected_log_prob, self.model_variables())
        grad_vector_product = tf.reduce_sum(log_prob_grad * vector)
        fisher_vector_product = -get_flattened_gradients(grad_vector_product, self.model_variables()) / states.shape[0]
        return self.session.run(fisher_vector_product, feed_dict={self.input_layer: states})        
    
    def compare_prob_ratios(self, states, actions, rewards, d_var):
        # Estimates the function to optimize as defined is section 5 of original paper
        # I take the ratio of probabilities from original weights and probs from proposed weights (var + d_var)
        # These ratios are weighted by Q-values: thus, if output > 1 new weights make better actions more probable
        # Also, wighout scaling by original probs, its gradient is given by grad_log_prob_actions
        action_mask = tf.one_hot(actions, depth=2, on_value=1.0, off_value=0.0, axis=-1)
        
        original_prob_actions = tf.reduce_sum(action_mask * self.prob_layer, axis=1)
        np_original_prob_actions = self.session.run(original_prob_actions, feed_dict={self.input_layer: states})

        for (grad, var) in zip(d_var, self.model_variables()):
            self.session.run(tf.assign_add(var, grad))
            
        new_prob_actions = tf.reduce_sum(action_mask * self.prob_layer, axis=1)   
        np_new_prob_actions = self.session.run(new_prob_actions, feed_dict={self.input_layer: states})

        for (grad, var) in zip(d_var, self.model_variables()):
            self.session.run(tf.assign_sub(var, grad))
            
        return sum((np_new_prob_actions / np_original_prob_actions) * np.array(rewards))

    def estimate_kl_divergence(self, states, d_var):
        # This function calculates an actual kl_divergence between action prob distributions
        # with current weights and weight + d_var
        original_probs = self.session.run(self.prob_layer, feed_dict={self.input_layer: states})
        original_log_probs = self.session.run(self.log_prob_layer, feed_dict={self.input_layer: states})

        for (grad, var) in zip(d_var, self.model_variables()):
            self.session.run(tf.assign_add(var, grad))
            
        new_log_probs = self.session.run(self.log_prob_layer, feed_dict={self.input_layer: states})

        for (grad, var) in zip(d_var, self.model_variables()):
            self.session.run(tf.assign_sub(var, grad))
            
        kl = original_probs * (original_log_probs - new_log_probs)
        return np.sum(kl) / kl.shape[0]
                

In [ ]:
class RL_Learner:
    
    def __init__(self, rl_agent, game_env, discount, batch_size):
        self.session = rl_agent.session
        self.agent = rl_agent
        self.env = game_env
        
        self.discount = discount
        self.batch_size = batch_size
        
        self.reward_history = []
        self.played_games = 0
        
    def play_single_game(self):
        states = None
        actions = []
        rewards = []
        
        observation = self.env.reset().reshape((1, 4))
        done = False
        
        while done == False:
            if states is None:
                states = observation
            else:
                states = np.concatenate((states, observation), axis=0)
            prob_actions = self.agent.predict(observation)[0]
            action = np.random.choice(np.arange(len(prob_actions)), p=prob_actions)
            actions.append(action)
            observation, reward, done, info = self.env.step(action)
            observation = observation.reshape((1, 4))
            rewards.append(reward)
            
        self.reward_history.append(sum(rewards))
        self.played_games += 1
            
        return states, actions, rewards
    
    def play_batch(self):        
        all_states = []
        all_actions = []
        all_rewards = []
        
        for i in range(self.batch_size):

            states, actions, rewards = self.play_single_game()
            
            all_states.append(states)
            all_actions.append(actions)
            all_rewards.append(sum_discounted_rewards(rewards, self.discount))
            
        print "Average reward for batch #", self.played_games / self.batch_size, \
              ": ", sum(self.reward_history[-self.batch_size:]) / self.batch_size
        
        concat_states = reduce(lambda x, y: np.concatenate((x, y), axis=0), all_states)
        concat_actions = reduce(lambda x, y: x + y, all_actions)
        concat_rewards = reduce(lambda x, y: x + y, all_rewards)     
        
        return concat_states, concat_actions, concat_rewards

In [ ]:
class PG_Learner(RL_Learner):
    
    def __init__(self, rl_agent, game_env, discount, batch_size, lr=0.1):
        RL_Learner.__init__(self, rl_agent, game_env, discount, batch_size)
        self.lr = lr
        
    def step(self):
        # One policy gradient step based on one batch of games
        
        concat_states, concat_actions, concat_rewards = self.play_batch()
        grad_reward = self.agent.pg_grad(concat_states,
                                         tf.constant(concat_actions),
                                         tf.constant(concat_rewards))  / self.batch_size
        
        grads = unflatten_gradient(tf.constant(self.lr * grad_reward, dtype=tf.float32), self.agent.model_variables())
        for (grad, var) in zip(grads, self.agent.model_variables()):
            self.session.run(tf.assign_add(var, grad))   

In [ ]:
tf.reset_default_graph()

pg = PG_Learner(rl_agent=RL_Agent("cartpole"), 
                game_env=env,
                discount=0.95, 
                batch_size=100, 
                lr=0.01)

for i in range(100):
    pg.step()
    
plt.plot(range(len(pg.reward_history)), pg.reward_history)
plt.show()

In [ ]:
class TRPO_Learner(RL_Learner):
    
    def __init__(self, rl_agent, game_env, discount, batch_size, trpo_delta=0.01, line_search_option="max"):
        RL_Learner.__init__(self, rl_agent, game_env, discount, batch_size)
        self.trpo_delta = trpo_delta
        # Line search options could be: 
        # * "none": trpo step is taken without checking actual KL-divergence
        # * "max": trpo step is maximal in picked direction that satisfies KL-constraint
        # * "best": trpo step is the one that satisfies the constraint and gives the best prob ratio along the line
        self.line_search_option = line_search_option
        
    def scale_down_grads(self, obs_states, grads):
        # This function adjusts trpo direction until it starts to satisfy KL-constraint
        while self.agent.estimate_kl_divergence(obs_states, grads) > self.trpo_delta:
            grads = [0.5 * grad for grad in grads]
        return grads
    
    def line_search(self, obs_states, obs_actions, obs_rewards, grads):
        # This function shrinks grads in various ways and picks the scaling that:
        # * satisfies KL-constraint
        # * gives the best weighted action prob ratio
        deltas = [2 ** (-i) for i in range(10)] + [0]

        def feasibility_check(delta):
            trunc_grads = [delta * grad for grad in grads]
            return self.agent.estimate_kl_divergence(obs_states, trunc_grads) <= self.trpo_delta
        deltas = filter(feasibility_check, deltas)
            
        def prob_ratio(delta):
            trunc_grads = [delta * grad for grad in grads]
            return self.agent.compare_prob_ratios(obs_states, obs_actions, obs_rewards, trunc_grads)
        
        best_delta = max(deltas, key=prob_ratio)
        return [best_delta * grad for grad in grads]
            
    def step(self):
        
        concat_states, concat_actions, concat_rewards = self.play_batch()
        
        grad_reward = self.agent.trpo_grad(concat_states,
                                           tf.constant(concat_actions),
                                           tf.constant(concat_rewards))  / self.batch_size
        
        Ax_fun = lambda x: self.agent.fisher_vector_product(concat_states, tf.constant(x, dtype=tf.float32))
        
        trpo_dir = cg_solver(Ax_fun, grad_reward)
        scaling = np.sqrt(2 * self.trpo_delta / np.sum(trpo_dir * Ax_fun(trpo_dir)))
        
        grads = unflatten_gradient(tf.constant(scaling * trpo_dir, dtype=tf.float32), self.agent.model_variables())
               
        if self.line_search_option == "max":
            grads = self.scale_down_grads(concat_states, grads)
        elif self.line_search_option == "best":
            grads = self.line_search(concat_states, concat_actions, concat_rewards, grads)  
                
        for (grad, var) in zip(grads, self.agent.model_variables()):
            self.session.run(tf.assign_add(var, grad))


In [ ]:
tf.reset_default_graph()

trpo = TRPO_Learner(rl_agent=RL_Agent("cartpole"), 
                    game_env=env,
                    discount=0.95, 
                    batch_size=100, 
                    trpo_delta=0.01,
                    line_search_option="max")

for i in range(100):
    trpo.step()
    
plt.plot(range(len(trpo.reward_history)), trpo.reward_history)
plt.show()